In [29]:
import pandas as pd
import re

In [81]:
file_path = '/Users/vittoriomanfriani/Desktop/bond_data.xlsx'
excel_data = pd.ExcelFile(file_path)
df_raw = excel_data.parse('Sheet1', header=None)

In [82]:
df_raw

,0,1,2,3,4,5,6,7,8,9,...,6494,6495,6496,6497,6498,6499,6500,6501,6502,6503
0,US91282CKQ32 ISIN,NaN,NaN,NaN,US91282CLF67 ISIN,NaN,NaN,NaN,US91282CLK52 ISIN,NaN,...,NaN,NaN,US912828WQ91 ISIN,NaN,NaN,NaN,US912828YC87 ISIN,NaN,NaN,NaN
1,date,price,yield mid,dv01 mid,date,price,yield mid,dv01 mid,date,price,...,yield mid,dv01 mid,date,price,yield mid,dv01 mid,date,price,yield mid,dv01 mid
2,2024-05-09 00:00:00,99.367188,4.454,7.969,2024-08-08 00:00:00,99.0625,3.99,8.135,2024-08-29 00:00:00,99.808594,...,0.927,2.947,2014-06-25 00:00:00,100.035156,0.482,1.988,2019-08-28 00:00:00,99.994141,1.503,1.955
3,2024-05-10 00:00:00,99.023438,4.497,7.936,2024-08-09 00:00:00,99.460938,3.941,8.173,2024-08-30 00:00:00,99.644531,...,0.921,2.947,2014-06-26 00:00:00,100.070312,0.465,1.989,2019-08-29 00:00:00,99.957031,1.522,1.954
4,2024-05-13 00:00:00,99.101562,4.487,7.944,2024-08-12 00:00:00,99.75,3.905,8.201,2024-09-02 00:00:00,99.644531,...,0.929,2.946,2014-06-27 00:00:00,100.074219,0.463,1.989,2019-08-30 00:00:00,99.986328,1.507,1.955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Extract the header row to identify ISINs and their column positions
header = df_raw.iloc[0]
header = [val.split()[0] if isinstance(val, str) and "ISIN" in val else val for val in header]

In [85]:
# Identify ISIN positions and their corrected names
isin_positions = [(i, val) for i, val in enumerate(header) if isinstance(val, str) and re.match(r'^[A-Z0-9]{12}$', val)]

In [86]:
# Function to convert only numeric dates
def convert_to_datetime(val):
    if isinstance(val, (int, float)):  # Check if the value is numeric
        return pd.to_datetime(val, origin='1899-12-30', unit='D')
    elif isinstance(val, str):  # Check if the value is already a date string
        return pd.to_datetime(val, errors='coerce')  # Convert if it's a valid date
    return val  # Return as-is if it doesn't match above cases

In [91]:
# Process each ISIN section and collect data
data_frames = []
for start_col, isin in isin_positions:
    # Determine the end column for each ISIN section (4 columns per ISIN)
    end_col = start_col + 4
    temp_df = df_raw.iloc[2:, start_col:end_col]  # Skip the first two rows (headers)
    temp_df.columns = ["date", "mid_price", "yield", "mid_dv01"]
    temp_df["isin"] = isin
    
    temp_df['date'] = temp_df['date'].apply(convert_to_datetime)
    
    temp_df.dropna(subset=['date'], inplace=True)  # Drop rows with any remaining invalid dates
    
    data_frames.append(temp_df)
    print(start_col)

In [92]:
# Concatenate all ISIN data into a single DataFrame
final_df = pd.concat(data_frames, ignore_index=True)
final_df.set_index(["date", "isin"], inplace=True)
final_df = final_df.sort_index()

In [96]:
final_df.to_csv('USBond_Dataset_Ordered.csv')